IBM - Semana 4 Datos - Marisa Chaves 


Datos del proyecto
Los restaurantes Japoneses en Madrid

Necesitamos datos sobre los restaurantes japoneses que hay en Madrid Capital por barrios para decidir dónde puede ser interesante montar uno, para ello:

Dataframe con identificadores, municipio, latitud y longitud de los barrios de Madrid:
Los datos del Dataframe salen de Google respecto a latitudes y longitudes y del ayuntamiento de Madrid para conocer todos los barrios de la ciudad de Madrid, Creamos un archivo excel con toda la información 
Pueden consultarse en Github Excell sobre Barrios Madrid y longitudes y Latitudes en el archivo Barrios Madrid CSV 

Información de los restaurantes de la zona de Madrid a través de una conexión a foursquare.

In [1]:
import numpy as np # librería para manejar datos vectorizados
import random # librería para generar números aleatorios
import pandas as pd # librería para análisis de datos
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json # librería para manejar archivos JSON 
!pip install geopy
#!conda install -c conda-forge geopy --yes # retirar el comentario de esta línea si no ha completado el laboratorio de la API de FourSquare 
from geopy.geocoders import Nominatim # convertir una dirección en valores de latitud y longitud
import requests # librería para manejar solicitudes
from pandas.io.json import json_normalize # librería para convertir un archivo json en un dataframe pandas
# Matplotlib y módulos asociados para graficar
import matplotlib.cm as cm
import matplotlib.colors as colors
# importar k-means desde la fase de agrupación
from sklearn.cluster import KMeans
! pip install folium==0.5.0
#!conda install -c conda-forge folium=0.5.0 --yes # retirar el comentario de esta línea si no ha completado el laboratorio de la API de FourSquare
import folium # librería para graficar mapas 
# librerías para mostrar imágenes 
from IPython.display import Image 
from IPython.core.display import HTML 
print('Libraries imported.')

Libraries imported.


In [2]:
import pandas as pd
archivo = 'Barrios Madrid, col 3.xlsx'
df = pd.read_excel(archivo, sheet_name='Hoja1')
df

,Barrio de Madrid,Latidude,Longitude
0,Embajadores,4040872527791480,-3702500204470750
1,Cortes,40414001032961400,-3699145349930450
2,Justicia,40423921537121904,-36980783956712704
3,Universidad,4042621886908230,-37095375308933504
4,Sol,4041663940530680,-3704062882021400
5,Imperial,4040496442574320,-37170254740668400
6,Acacias,40402454249880000,-37056396791731
7,Chopera,403944098725109,-36998134121444896
8,Legazpi,4039041942879880,-3691665580933380
9,Delicias,40400168393429296,-3690371113475480


In [3]:
address = 'Madrid, España'
geolocator = Nominatim(user_agent="Madrid")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Madrid are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Madrid are 40.4167047, -3.7035825.


In [5]:
# conexión a foursquare
CLIENT_ID = '5DVDSUQ2CTJVYAHPUPQBNQF5FOKPDZ4O1J0M0XQ00GZYX2DN'
CLIENT_SECRET = 'EGZAZEZ2HPWF1YL2FKDWFFMEE1DDFTZ0PD2GUS5PK3SB4U4E'
VERSION = 20220221
LIMIT = 1000